In [32]:
import logging
import os

import wiki as w 
 
from localgensim.gensim2.models import word2vec 
from gensim.models import KeyedVectors

import numpy as np
import sys

from tqdm import tqdm

In [9]:
w.download_wiki_dump('en', '/home/manni/data/wiki/en-28102022.bz2')

[2022-09-28 12:48:57,898] Downloading https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles-multistream.xml.bz2 to /home/manni/data/wiki/en-28102022.bz2
 63%|██████████████████████████████████████████████████████████████████████████████████████████████████▏                                                        | 13.1M/20.7M [46:22<26:27, 4.78kKB/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 79%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                | 16.4M/20.7M [57:36<15:12, 4.73kKB/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order 

In [105]:
import sys
sys.path.append("../../imports/")
import saver as sv

In [11]:
logging.basicConfig(format='[%(asctime)s] %(message)s', level=logging.INFO)
os.makedirs('data/', exist_ok=True)

In [14]:
sentences = sv.load("wiki_sentences_wnet_noun") #New

In [198]:
def get_sents(word,n):
    sents = list()
    _n = 1
    for sent in sentences:
        for i,_word in enumerate(sent):
            if word == _word or word+'#' in _word or word+'§' in _word:
                sents.append(' '.join(sent[i-10:i]+sent[i:i+11]))
                _n+=1
                if _n>n:
                    return sents
                break

In [192]:
#happy cheerful
#smart intelligent
#hard difficult
#large huge
word = 'bank'

In [199]:
sents=get_sents(word,5)

In [200]:
for i,sent in enumerate(sents):
    print(i+1,sent)

1 association voluntary contract federation monetary reform credit currency would regulated bank people§ mutualism ha retrospectively characterised ideologically situated individualist collectivist#1 form#3
2 well fargo tower birmingham#1 financial§ district#0§ region#2§ financial corporation largest bank headquartered operating#1 alabama pnc financial service§ well fargo also major
3 black troop#0 lincoln§ wrote bare sight armed drilled black soldier§ bank mississippi would end rebellion end lincoln direction#5§ general lorenzo thomas§
4 nest polglase goldwyn folly richard day holiday stephen goosson lionel bank king han dreier john b goodman mad music jack otterson
5 area§ around#0 living rural area§ access#3§ improved sanitation according world bank algeria making#24 progress#1 toward goal§ reducing#11 half number§ people without


In [34]:
#w2v = '/home/manni/embs/w2v.bin.gz'
import gensim.downloader as api

w2v = api.load('word2vec-google-news-300')

[2022-09-30 10:25:57,635] Creating /home/manni/gensim-data


[--------------------------------------------------] 1.4% 23.4/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=-------------------------------------------------] 3.4% 55.9/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=================================================-] 98.1% 1631.0/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=================================================-] 99.5% 1654.7/1662.8MB downloaded


[2022-09-30 10:28:17,113] word2vec-google-news-300 downloaded
[2022-09-30 10:28:35,265] loading projection weights from /home/manni/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
[2022-09-30 10:31:18,694] loaded (3000000, 300) matrix from /home/manni/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [205]:
model  = w2v
vocab = model.vocab

In [219]:
word2desc = sv.load('word2desc')

In [206]:
def sense_vec(word,model):
    v = list()
    if word not in word2desc:
        return [np.zeros(300)]
    for words in word2desc[word]:
        if not words:
            continue
        _v = list()
        for _word in words:
            if _word in model.vocab:
                _v.append(model.get_vector(_word))
        if not _v:
            continue
        _v = np.sum(_v,axis=0)
        v.append(_v)
    return v

In [207]:
sense_vec(word,model)

[]

In [201]:
def tag_sense(word,sents):
    tgs = list()
    for sent in sents:
        sent = [word.split('#')[0].split('§')[0] for word in sent.split()]
        i = sent.index(word)
        target = sent[i]
        context = sent[i-3:i]+sent[i+1:i+4]
        tgs.append((target,context))
    return tgs

In [221]:
tgs = tag_sense(word,sents)

In [181]:
word2desc_raw=sv.load('worddesc_raw')

In [222]:
for target,context in tgs:
    print('Context words:', context,r'\\')
    print(r'------------\\')
    svec = sense_vec(target,model)
    maxi = 0
    res = None
    for i,v in enumerate(svec):
        for con in context:
            if con == target:
                continue
            _svec = sense_vec(con,model)
            for j,_v in enumerate(_svec):
                sim = model.cosine_similarities(v, [_v])[0]
                if sim>maxi:
                    maxi = sim
                    #res = (i,con,j)
                    res = ' Target: {1}{0}\n Target sense: {2}{0}\n Context: {3}{0}\n Context sense: {4}{0}\n '\
                    .format(r'\\',target,word2desc_raw[target][i],con,word2desc_raw[con][j])
    print(res)
    print(r'------------\\')

Context words: ['currency', 'would', 'regulated', 'people', 'mutualism', 'ha'] \\
------------\\
 Target: bank\\
 Target sense: a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force\\
 Context: regulated\\
 Context sense: shape or influence; give direction to\\
 
------------\\
Context words: ['financial', 'corporation', 'largest', 'headquartered', 'operating', 'alabama'] \\
------------\\
 Target: bank\\
 Target sense: a financial institution that accepts deposits and channels the money into lending activities\\
 Context: corporation\\
 Context sense: a business firm whose articles of incorporation have been approved in some state\\
 
------------\\
Context words: ['drilled', 'black', 'soldier', 'mississippi', 'would', 'end'] \\
------------\\
 Target: bank\\
 Target sense: a flight maneuver; aircraft tips laterally about its longitudinal axis (especially in turning)\\
 Context: end\\
 Context sense: the surfa

In [215]:
word2desc['happy']

[{'enjoying', 'joy', 'pleasure', 'showing'},
 {'fortune', 'good'},
 {'act', 'disposed', 'eagerly', 'service'},
 {'expressed', 'point', 'well'}]

In [ ]:
word2desc['rapid']

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
rapid = wn.synsets('rapid')
print(rapid)

In [ ]:
for i in rapid:
    print(i.hyponyms())

In [ ]:
for i in rapid:
    print(i.hypernyms())

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
glove = '/home/manni/embs/gloveW2V.6B.300d.txt'
w2v = '/home/manni/embs/w2v.bin.gz'
spx = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_mixture.txt'
wnet ='/home/manni/embs/en_wiki_wnet_epoch5_300_w10_exclusive.txt'

In [ ]:
w2v_model = KeyedVectors.load_word2vec_format(w2v, binary=True)

In [ ]:
glove_model = KeyedVectors.load_word2vec_format(glove, binary=False)

In [ ]:
spx_model = KeyedVectors.load_word2vec_format(spx, binary=False)

In [ ]:
wnet_model = KeyedVectors.load_word2vec_format(wnet, binary=False)

In [ ]:
#model = glove_model

In [ ]:
#model = w2v_model

In [ ]:
#model = spx_model

In [ ]:
model = wnet_model

In [ ]:
def get_matrix(model,vocab):
    matrix = list()
    for word in vocab:
        if word in model.vocab:
            vec = model.get_vector(word)
        else:
            vec = np.zeros(model.vector_size)
        matrix.append(vec)
    return np.asarray(matrix)

In [ ]:
def get_clusters(model,word):
    sets = word2desc[word]
    clusters_ = list()
    n_clusters_ = list()
    for words in sets:
        vocab = list(words)
        matrix = get_matrix(model,vocab)
        db = DBSCAN(eps=0.4,min_samples=2,metric='cosine').fit(matrix)
        labels = db.labels_
        clusters = dict()
        for i,c in enumerate(labels):
            if c == -1:
                continue
            if c in clusters:
                clusters[c].append(vocab[i])
            else:
                clusters[c] = [vocab[i]]
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        clusters_.append(clusters)
        n_clusters_.append(n_clusters)
    return clusters_,n_clusters_

In [ ]:
get_clusters(model,'rapid')

In [ ]:
for words in word2desc['rapid']:
    words = list(words)
    for i in range(len(words)):
        for j in range(i+1,len(words)):
            print(words[i],words[j],model.similarity(words[i],words[j]))
    print('------')

In [ ]:
model.similarity(['waterway','river'],'current')

In [ ]:
model.doesnt_match({'current', 'fast', 'part', 'river', 'waterway'})

In [ ]:
v = model.get_vector('waterway')+model.get_vector('river')

In [ ]:
model.cosine_similarities(v,[model.get_vector('current')])

In [ ]:
model

In [ ]:
a = ['current#3','current#2','current#1','current','current#0']
b = ['river','river#0']
c = ['waterway','waterway#0','waterway#1']

In [ ]:
for word in a:
    for _word in b:
        print(word,_word,model.similarity(word,_word))
    print('-------')

In [ ]:
def avg_sim(w1,w2,model,verbose=False,maxsim=False):
    range_ = list()
    vocab = model.vocab
    a = list()
    for word in vocab:
        if word.startswith(w1+'#'):
            a.append(word)
    if w1 in vocab:
        a.append(w1)
    b = list()
    for word in vocab:
        if word.startswith(w2+'#'):
            b.append(word)
    if w2 in vocab:
        b.append(w2)
    div = len(a)*len(b)
    if div == 0:
        return 0, range_
    sims = list()
    for i in a:
        for j in b:
            sim = model.similarity(i,j)
            range_.append(sim)
            if verbose:
                print(i,j,sim)
            sims+=[sim]
    if maxsim:
        return max(sims), range_
    else:
        return sum(sims)/div, range_

In [ ]:
avg_sim('rapid','fast',model,verbose=True,maxsim=True)

# tags

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
doc = nlp("the raft floated downstream on the current")
for token in doc:
    #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
    print(token.pos_)

# contexts

In [4]:
sentences = sv.load("wiki_sentences_wnet")

In [5]:
for sent in sentences:
    print(sent[:100])
    break

FileNotFoundError: [Errno 2] No such file or directory: '/home/manni/data/wiki/enwiki-20211120-pages-articles-multistream.xml.bz2'

In [ ]:
examples = list()
def get_contexts(word,sentences):
    i = 1
    for sent in sentences:
        for j,_word in enumerate(sent):
            if word == _word.split('#'):
                examples.append(sent[j-10:j]+sent[j:j+11])
                i+=1
            elif word == _word.split('§'):
                examples.append(sent[j-10:j]+sent[j:j+11])
                i+=1
            if i>10:
                break
        if i>10:
                break
    return examples

In [ ]:
get_contexts('rapid',sentences)